2021UCA1809
2021UCA1848

Vehicle and License Plate Detection

# Installing and Importing libraries

In [ ]:
!pip install easyocr
!pip install ultralytics
!pip install filterpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.3/716.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=ec0aa81a024c8840ed4c68ef0e97e94d47feb79256884d8a68dca49e78cad517
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy


In [ ]:
import filterpy
import string
import easyocr
from ultralytics import YOLO
import cv2
import csv
import numpy as np

# Function definitions

In [ ]:
# Initialize the OCR reader
reader = easyocr.Reader(['en'], gpu=True)

# Mapping dictionaries for character conversion
dict_char_to_int = {'O': '0', 'I': '1', 'J': '3', 'A': '4', 'G': '6', 'S': '5'}
dict_int_to_char = {'0': 'O', '1': 'I', '3': 'J', '4': 'A', '6': 'G', '5': 'S'}

In [ ]:
def write_csv(results, output_path):
    aggregated_results = {}

    for car_id, data in results.items():
        text = data['text']
        text_score = data['text_score']

        if text in aggregated_results:
            aggregated_results[text] = max(aggregated_results[text], text_score)
        else:
            aggregated_results[text] = text_score

    with open(output_path, 'w', newline='') as csv_file:
        fieldnames = ['text', 'score']
        csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        csv_writer.writeheader()

        for text, score in aggregated_results.items():
            if score < 0.25:
                continue
            csv_writer.writerow({'text': text, 'score': score})

The function write the results to a CSV file in a (text, confidence score) format.

    Args:
        results (dict): Dictionary containing the results.
        output_path (str): Path to the output CSV file.

In [ ]:
def license_complies_format(text):
    if len(text) != 7:
        return False

    if (text[0] in string.ascii_uppercase or text[0] in dict_int_to_char.keys()) and \
       (text[1] in string.ascii_uppercase or text[1] in dict_int_to_char.keys()) and \
       (text[2] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[2] in dict_char_to_int.keys()) and \
       (text[3] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()) and \
       (text[4] in string.ascii_uppercase or text[4] in dict_int_to_char.keys()) and \
       (text[5] in string.ascii_uppercase or text[5] in dict_int_to_char.keys()) and \
       (text[6] in string.ascii_uppercase or text[6] in dict_int_to_char.keys()):
        return True
    else:
        return False

The function checks if the license plate text complies with the required format.

    Args:
        text (str): License plate text.

    Returns:
        bool: True if the license plate complies with the format, False otherwise.

In [ ]:
def format_license(text):
    license_plate_ = ''
    mapping = {0: dict_int_to_char, 1: dict_int_to_char, 4: dict_int_to_char, 5: dict_int_to_char, 6: dict_int_to_char,
               2: dict_char_to_int, 3: dict_char_to_int}
    for j in [0, 1, 2, 3, 4, 5, 6]:
        if text[j] in mapping[j].keys():
            license_plate_ += mapping[j][text[j]]
        else:
            license_plate_ += text[j]

    return license_plate_

The function formats the license plate text by converting characters using the mapping dictionaries.

    Args:
        text (str): License plate text.

    Returns:
        str: Formatted license plate text.

In [ ]:
def read_license_plate(license_plate_crop):
    detections = reader.readtext(license_plate_crop)

    for detection in detections:
        bbox, text, score = detection

        text = text.upper().replace(' ', '')

        if license_complies_format(text):
            return format_license(text), score

    return None, None

The function reads the license plate text from the given cropped image (if it exists).

    Args:
        license_plate_crop (PIL.Image.Image): Cropped image containing the license plate.

    Returns:
        tuple: Tuple containing the formatted license plate text and its confidence score.

In [ ]:
def get_car(license_plate, vehicle_ids):
    x1, y1, x2, y2, score, class_id = license_plate

    foundIt = False
    for j in range(len(vehicle_ids)):
        xcar1, ycar1, xcar2, ycar2, car_id = vehicle_ids[j]

        if x1 > xcar1 and y1 > ycar1 and x2 < xcar2 and y2 < ycar2:
            car_indx = j
            foundIt = True
            break

    if foundIt:
        return vehicle_ids[car_indx]

    return -1, -1, -1, -1, -1

The function helps retrieve vehicle coordinates and vehicle ID based on the license plate coordinates.

    Parameters:
        license_plate (tuple): Tuple containing the coordinates of the license plate (x1, y1, x2, y2, score, class_id).
        vehicle_ids (list): List of vehicle track IDs and their corresponding coordinates.

    Returns:
        tuple: Tuple containing the vehicle coordinates (x1, y1, x2, y2) and ID.

# SORT Algorithm

We use the [SORT algorithm](https://github.com/abewley/sort) for 2D multiple object tracking in video sequences. For that we first upload the module and unzip it and then import it.

In [ ]:
!unzip -q my_sort

In [ ]:
from my_sort.my_sort import *

# Vehicle Detection

Using the trained COCO model, SORT algorithm and the functions defined above earlier to pass a video recording and to obtain car license plates in the CSV file.

In [ ]:
coco_model = YOLO('./yolov8n.pt')
detector = YOLO('./best.pt')

vehicles = [2, 3, 5, 7]

In the COCO Model, label 2 represents car, 3 represents motorbike, 5 represents bus and 7 represents truck.

In [ ]:
results = {}

tracker = Sort()

In [ ]:
cap = cv2.VideoCapture('./sample.mp4')

In [ ]:
fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)

60.0


In [ ]:
frame_no = -1
ret = True
while ret:
    frame_no += 1
    ret, frame = cap.read()
    if ret:
        # detect vehicles
        detections = coco_model(frame)[0]
        detections_ = []
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = detection
            if int(class_id) in vehicles:
                detections_.append([x1, y1, x2, y2, score])

        # track vehicles
        ids = tracker.update(np.asarray(detections_))

        # detect license plates
        license_plates = detector(frame)[0]
        for license_plate in license_plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = license_plate

            # assign license plate to car
            xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, ids)

            if car_id != -1:

                # crop license plate
                license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]

                # process license plate
                license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
                _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)

                # read license plate number
                license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop_thresh)

                if license_plate_text is not None:
                    if car_id not in results or results[car_id]['text_score'] < license_plate_text_score:
                        results[car_id] = {'text': license_plate_text, 'text_score': license_plate_text_score}

Streaming output truncated to the last 5000 lines.
0: 384x640 1 License_Plate, 11.2ms
Speed: 2.9ms preprocess, 11.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 2 trucks, 13.3ms
Speed: 3.7ms preprocess, 13.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 11.0ms
Speed: 3.2ms preprocess, 11.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 2 trucks, 25.8ms
Speed: 10.7ms preprocess, 25.8ms inference, 5.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 17.8ms
Speed: 3.7ms preprocess, 17.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 cars, 1 bus, 2 trucks, 25.2ms
Speed: 3.4ms preprocess, 25.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.6ms
Speed: 4.3ms preprocess, 13.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 3

In [ ]:
write_csv(results, './test.csv')

In [ ]:
results

{9553.0: {'text': 'NA13NRU', 'text_score': 0.6753512130346838},
 9555.0: {'text': 'MV51VSU', 'text_score': 0.33077608216842574},
 9556.0: {'text': 'GX15OGJ', 'text_score': 0.7508134502699505},
 9563.0: {'text': 'LM13VCV', 'text_score': 0.3566346137043862},
 9558.0: {'text': 'KH05ZIK', 'text_score': 0.7907690682979504},
 9551.0: {'text': 'AP05JEO', 'text_score': 0.6892391060688103},
 9565.0: {'text': 'FJ14ZHY', 'text_score': 0.9033641831481448},
 10052.0: {'text': 'EY61NBG', 'text_score': 0.9252822440657257},
 10380.0: {'text': 'BG65USJ', 'text_score': 0.612603311819423},
 10396.0: {'text': 'BG65USJ', 'text_score': 0.728703082281836},
 10441.0: {'text': 'BG65USJ', 'text_score': 0.586427208750937},
 10418.0: {'text': 'BG65USJ', 'text_score': 0.8401685656757607},
 10082.0: {'text': 'AK64DMV', 'text_score': 0.9506085373907489},
 10492.0: {'text': 'EG65USJ', 'text_score': 0.49228977345683833},
 10501.0: {'text': 'EF10DZT', 'text_score': 0.8520065728618018},
 10225.0: {'text': 'AF65JKV', 'te